## 1. Experimental Setup
1.1 Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1.2 Bioconda setup

In [ ]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 76 (delta 26), reused 59 (delta 15), pack-reused 0
Receiving objects: 100% (76/76), 318.16 KiB | 8.84 MiB/s, done.
Resolving deltas: 100% (26/26), done.
./
./root/
./root/.bashrc.biolab
./root/.bin.priority/
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.bin.priority/pip3
./root/.condarc
./root/.profile
./root/.tmux.conf
./root/.vimrc
--2024-05-30 00:28:29--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143808873 (137M) [application/octet-stream]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>] 137.15M  4

1.3 Install *samtools* for analysis

In [ ]:
!conda install -y bedtools bioawk samtools

Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: / - \ done

## Package Plan ##

  environment location: /root/conda

  added / updated specs:
    - bedtools
    - bioawk
    - samtools


The following packages will be UPDATED:

  conda              pkgs/main::conda-24.4.0-py312h06a4308~ --> conda-forge::conda-24.5.0-py312h7900ff3_0 
  libgcc-ng          pkgs/main::libgcc-ng-11.2.0-h1234567_1 --> conda-forge::libgcc-ng-13.2.0-h77fa898_7 
  libgomp              pkgs/main::libgomp-11.2.0-h1234567_1 --> conda-forge::libgomp-13.2.0-h77fa898_7 
  libstdcxx-ng       pkgs/main::libstdcxx-ng-11.2.0-h12345~ --> conda-forge::libstdcxx-ng-13.2.0-hc0a3c3a_7 
  ncurses                 pkgs/main::ncurses-6.4-h6a678d5_0 --> conda-forge::ncurses-6.5-h59595ed_0 
  openssl              pkgs/main::openssl-3.0.13-h7f8727e_1 --> conda-forge::openssl-3.3.0-h4ab18f5_3 

The following packages will be SUPERSEDED by a higher-priority channel:

  _libgcc_mutex      

1.4 Acess the given sequencing data


In [ ]:
!mkdir -p /content/drive/MyDrive/binfo1-work
%cd /content/drive/MyDrive/binfo1-work
!cp ../binfo1-datapack1/* .

In [ ]:
%cd /content/drive/MyDrive/binfo1-work/

/content/drive/MyDrive/binfo1-work


## 2. Find the Lin28a binding site
Search the coordinates of the gene whose "transcript_support_level" is 1 (chr4, 133730641-133746152)

In [181]:
!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files


In [ ]:
!grep -i lin28a ../binfo1-datapack1/gencode.gtf

chr4	HAVANA	gene	133730641	133746152	.	-	.	gene_id "ENSMUSG00000050966.10"; gene_type "protein_coding"; gene_name "Lin28a"; level 2; mgi_id "MGI:1890546"; havana_gene "OTTMUSG00000010211.3";
chr4	HAVANA	transcript	133730641	133746152	.	-	.	gene_id "ENSMUSG00000050966.10"; transcript_id "ENSMUST00000051674.3"; gene_type "protein_coding"; gene_name "Lin28a"; transcript_type "protein_coding"; transcript_name "Lin28a-201"; level 2; protein_id "ENSMUSP00000050488.3"; transcript_support_level "1"; mgi_id "MGI:1890546"; tag "basic"; tag "appris_principal_1"; tag "CCDS"; ccdsid "CCDS18761.1"; havana_gene "OTTMUSG00000010211.3"; havana_transcript "OTTMUST00000023733.1";
chr4	HAVANA	exon	133746022	133746152	.	-	.	gene_id "ENSMUSG00000050966.10"; transcript_id "ENSMUST00000051674.3"; gene_type "protein_coding"; gene_name "Lin28a"; transcript_type "protein_coding"; transcript_name "Lin28a-201"; exon_number 1; exon_id "ENSMUSE00000361592.4"; level 2; protein_id "ENSMUSP00000050488.3"; transcript_su

Extract a samfile where reads are located around the coordinates

In [ ]:
!samtools view -b -o Lin28a.bam ../binfo1-datapack1/CLIP-35L33G.bam chr4:133730641-133746152
!samtools view Lin28a.bam | wc -l

4293


In [ ]:
!samtools mpileup Lin28a.bam > Lin28a.pileup
!wc -l Lin28a.pileup

[mpileup] 1 samples in 1 input files
144525 Lin28a.pileup


In [ ]:
!awk '$2 >= 133730641 && $2 <= 133746152 { print $0; }' Lin28a.pileup > Lin28a-gene.pileup
!head Lin28a-gene.pileup

chr4	133730641	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730642	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730643	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730644	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730645	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730646	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730647	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730648	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730649	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB
chr4	133730650	N	16	<<<<<<<<<<<<<<<<	IGIGGIH=EHGIHIDB


Remove sequencing tags and consider only plus strand

In [ ]:
import pandas as pd
import re

pileup = pd.read_csv('Lin28a-gene.pileup', sep='\t', names=['chr', 'pos', '_ref', 'count', 'basereads', 'quals'])
toremove = re.compile('[<>$*#^]')
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

pileup[['chr', 'pos', 'matches']]

,chr,pos,matches
0,chr4,133730641,
1,chr4,133730642,
2,chr4,133730643,
3,chr4,133730644,
4,chr4,133730645,
...,...,...,...
15507,chr4,133746148,
15508,chr4,133746149,
15509,chr4,133746150,
15510,chr4,133746151,


Calculate error frequency of each base position using Shannon's Entropy

In [ ]:
import math

countDict = {}
rc = {'A':'t', 'C':'g', 'G':'c', 'T':'a'}
for pos in pileup['pos']:
  baseCount = {'A':0, 'C':0, 'G':0, 'T':0}
  for base in baseCount:
    baseCount[base] = pileup[pileup['pos'] == pos].iloc[0]['matches'].count(base)
  mbaseCount = {'a':0, 'c':0, 'g':0, 't':0}
  for base in mbaseCount:
    mbaseCount[base] = pileup[pileup['pos'] == pos].iloc[0]['matches'].count(base)
  for base in baseCount:
    baseCount[base] += mbaseCount[rc[base]]
  countDict[pos] = baseCount

In [ ]:
entropyDict = {}
for pos in countDict:
  total = sum(list(countDict[pos].values()))
  entropy = -sum((count / total) * math.log2(count / total) for count in list(countDict[pos].values()) if count > 0)
  entropyDict[pos] = entropy

Filter "Entropy > 0.8" & "read depth > 50" to find high-error rate areas, which can be a candidate of most-frequently cross-linked site

In [ ]:
POS = []
for pos in countDict:
  if sum(list(countDict[pos].values())) > 50 and entropyDict[pos] > 0.8:
    POS.append(pos)

for pos in POS:
  print(pos, max(countDict[pos], key=countDict[pos].get))

133730678 G
133730901 T
133730915 G
133731610 G
133731849 G
133731920 G
133732431 G
133732901 A
133732964 G
133733184 A
133735260 G
133735329 G
133735370 G
133735387 G


Find the binding motif corresponding to the high-error rate areas from the UCSC reference genome

Check whether the Lin28A binding motif, "AAGNHG", is contained in the motif candidates

*H: A, C, or U

## 3. Visualize